In [1]:
import os
import pandas as pd
import torch

import matplotlib.pyplot as plt
import numpy as np

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [2]:
from torch.utils.data import DataLoader, Dataset
data=pd.read_csv('C:/Users/Ishit/Desktop/birds_rev2/birds.csv')
bird_family= pd.read_csv('C:/Users/Ishit/Desktop/birds_rev2/class_dict.csv')
train_data = data[data['data set']=='train'][['filepaths', 'labels']]
valid_data = data[data['data set']=='valid'][['filepaths', 'labels']]
test_data = data[data['data set']=='test'][['filepaths', 'labels']]
valid_data.iloc[0]
class Alexnetdataset(Dataset):
    def __init__(self, datafr, path, class_dict, transform = None):
        self.datafr = datafr
        self.path = path
        self.transform = transform
        self.class_dict = class_dict
    def __len__(self):
        return self.datafr.shape[0]
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        img_path = os.path.join(self.path, self.datafr.iloc[idx][0])
        img = plt.imread(img_path)
        label = self.class_dict[self.datafr.iloc[idx][1]]
        
        if self.transform :
            img = self.transform(img)
        return img, label

In [11]:
class_dict={}
for i in range(275):
    class_dict[bird_family.loc[i][1]] = i
new_dict = {}
for i in class_dict:
    new_dict[class_dict[i]] = i

In [4]:
import torchvision
train_dataset = Alexnetdataset(datafr = train_data, path = 'birds_rev2/', class_dict = class_dict, transform = torchvision.transforms.functional.to_tensor)
test_dataset  = Alexnetdataset(datafr = test_data, path = 'birds_rev2/', class_dict = class_dict, transform = torchvision.transforms.functional.to_tensor)
valid_dataset= Alexnetdataset(datafr = valid_data, path = 'birds_rev2/', class_dict = class_dict, transform = torchvision.transforms.functional.to_tensor)

In [9]:
from torch.nn import Module
import torch.nn.functional as F
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, num_classes=10):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(3,96,11,4)
            self.conv2 = nn.Conv2d(96,256,5,padding=2,groups=2)
            self.conv3 = nn.Conv2d(256,384,3,padding=1)
            self.conv4 = nn.Conv2d(384,384,3,padding=1, groups=2)
            self.conv5 = nn.Conv2d(384,256,3,padding=1, groups=2)
            self.fc1 = nn.Linear(256*6*6,4096)
            self.fc2 = nn.Linear(4096,1200)
            self.fc3 = nn.Linear(1200,275)
 
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)),(2,2))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(F.relu(self.conv5(x)),(2,2))
        x = x.view(x.size(0),256*6*6)
        x = F.dropout(F.relu(self.fc1(x)),p=0.5)
        x = F.dropout(F.relu(self.fc2(x)),p=0.5)
        x = self.fc3(x)
        return x

model = Net()
print(model)
if train_on_gpu:
    model.cuda()

Net(
  (conv1): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4))
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2), groups=2)
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2)
  (conv5): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2)
  (fc1): Linear(in_features=9216, out_features=4096, bias=True)
  (fc2): Linear(in_features=4096, out_features=1200, bias=True)
  (fc3): Linear(in_features=1200, out_features=275, bias=True)
)


In [10]:
epochs =10 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()
train_loader = DataLoader(train_dataset, batch_size = 15, shuffle = True)
validation_loader = DataLoader(valid_dataset, batch_size = 15, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 15, shuffle = True)
for e in range(epochs):
    train_loss = 0.
    validation_loss = 0.
    valid_right = 0
    tot_train = 0
    i = 0
    for x, y in train_loader:
        x, y = x.cuda(), y.cuda()
        optimizer.zero_grad()
        prediction = nn.functional.log_softmax(model(x), dim=1)
        loss = loss_fn(prediction, y)
        loss.backward()
        optimizer.step()        
        train_loss += loss.item()*x.size(0)
        tot_train += x.size(0)        
        x.cpu()
        torch.cuda.empty_cache()
        i += 1
        if i%250 == 0:
            validation_loss = 0
            valid_right = 0
            model.eval()
            for x, y in validation_loader:
                x, y = x.cuda(), y.cuda()
                prediction = model(x)
                loss = (loss_fn(prediction, y))
                validation_loss += loss.item()*x.size(0)
                valid_right += (nn.functional.softmax(prediction,dim = 1).argmax(1)==y).sum().item()
            print(f'Train loss: {train_loss/tot_train}, Valid loss: {validation_loss/len(validation_loader.dataset)}',f'Valid accuracy: {valid_right/len(validation_loader.dataset)*100}', sep = '\n')
            model.train()

Train loss: 5.61612691116333, Valid loss: 5.618093285994096
Valid accuracy: 0.2909090909090909
Train loss: 5.6162694597244265, Valid loss: 5.617561664581299
Valid accuracy: 0.36363636363636365
Train loss: 5.616176947911581, Valid loss: 5.6202578804709695
Valid accuracy: 0.2181818181818182
Train loss: 5.615155924320221, Valid loss: 5.62149825703014
Valid accuracy: 0.36363636363636365
Train loss: 5.614368057632446, Valid loss: 5.618117214549671
Valid accuracy: 0.36363636363636365
Train loss: 5.6143237415949505, Valid loss: 5.617853334600275
Valid accuracy: 0.36363636363636365
Train loss: 5.614391444887434, Valid loss: 5.618372575586492
Valid accuracy: 0.36363636363636365
Train loss: 5.61398072886467, Valid loss: 5.620440644350919
Valid accuracy: 0.14545454545454545
Train loss: 5.613728430642022, Valid loss: 5.622511203072288
Valid accuracy: 0.5090909090909091
Train loss: 5.613330438041687, Valid loss: 5.621490100513805
Valid accuracy: 0.36363636363636365
Train loss: 5.612492546428333, Va

ValueError: pic should be 2/3 dimensional. Got 0 dimensions.